In [12]:
# Dynamic Pricing Code
# Summer Analytics 2025

# ========================
# Install Required Libraries
# ========================
!pip install pathway bokeh panel --quiet

# ========================
# Import Libraries
# ========================
import pandas as pd
import pathway as pw
import bokeh.plotting
import panel as pn

# ========================
# Load & Preprocess Data in Pandas
# ========================
df = pd.read_csv('/content/dataset.csv')

df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                 format='%d-%m-%Y %H:%M:%S')

df['vehicle_type'] = df['Occupancy'].apply(lambda x: 'car' if x % 2 == 0 else 'bike')
df['traffic'] = 0.5
df['special_day'] = 0
df['QueueLength'] = df['Occupancy'] % 4

# ========================
# Calculate Model 1 Prices
# ========================
base_price = 10
alpha = 2
df['price_model1'] = base_price + alpha * (df['Occupancy'] / df['Capacity'])

# ========================
# Calculate Model 2 Prices
# ========================
alpha_d = 1
beta_d = 0.5
gamma_d = 0.3
delta_d = 1.2
epsilon_d = {'car': 0.2, 'bike': 0.1}

df['vehicle_weight'] = df['vehicle_type'].map(epsilon_d).fillna(0.1)

df['demand'] = (
    alpha_d * (df['Occupancy'] / df['Capacity'])
    + beta_d * df['QueueLength']
    - gamma_d * df['traffic']
    + delta_d * df['special_day']
    + df['vehicle_weight']
)

df['demand_norm'] = df['demand'] / (df['demand'].max() + 1e-5)
df['price_model2'] = base_price * (1 + 0.5 * df['demand_norm'])

# ========================
# Save Final Prices to CSV
# ========================
df[['Timestamp', 'Occupancy', 'Capacity', 'price_model1', 'price_model2']].to_csv('predicted_prices.csv', index=False)

# ========================
# Visualisation (Optional)
# ========================
pn.extension()

p = bokeh.plotting.figure(x_axis_type='datetime', width=800, height=400,
                          title="Pricing Models Comparison")
p.line(df['Timestamp'], df['price_model1'], color='blue', legend_label="Model 1 - Linear", line_width=2)
p.line(df['Timestamp'], df['price_model2'], color='green', legend_label="Model 2 - Demand-Based", line_width=2)
p.legend.location = "top_left"

bokeh.plotting.show(p)
